In [40]:
import os
import pickle
import numpy as np
import pandas as pd
import operate_data as od
import ml_model as ml
from  demo import Predictor
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import ipdb
from collections import Counter
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

VECTOR_MODE = {'onehot': 0, 'wordfreq': 1, 'twovec': 2, 'tfidf': 3, 'outofdict': 4}

## Run emo classifier , return dict and counter of

In [41]:
def emo_classifier(date_start='2016/09',date_end='2016/10',data='./data/China.csv'):
    

    news_df=pd.read_csv(data,encoding='cp950')
    news_df['Time'] = pd.to_datetime(news_df['Time'])  
    news_df.dropna(inplace=True)
    news_df.reset_index(drop=True, inplace=True)
    #mask = (df['date'] > start_date) & (df['date'] <= end_date)
    mask=(news_df['Time']>=date_start) &( news_df['Time']<date_end )
    news_df=news_df[mask]
 
    reload=False
    if reload:
            best_vector = "wordfreq"
            best_model = 1  # linearLogistic
            save_model(best_vector, best_model)
    else:
            od.loadStopwords()
            od.loadEmotionwords()
            od.loadWords(od.stopList)
            od.loadDocument(od.stopList)
            
    predictor=Predictor()
    predictor.load_model()
    predictor.set_mode(mode="wordfreq")
    some_new=[]
    tags=[]
    time=[]
    titles=[]
    result={}
    for date,title,news in zip(news_df['Time'],news_df['Title'],news_df['Content']):
        predictor.set_news(news=news)
        predictor.trans_vec()
        tag = predictor()
#         date=datetime.strptime(date,"%Y/%m/%d")
   
        date.strftime('%Y/%m/%d')
        time.append(date)
        titles.append(title)
        some_new.append(news)
        tags.append(tag[0])
       
    counter=Counter(tags)
    result={
            'Time':time,
            'Title':titles,
            'Content':some_new,
            'Tag':tags
    }
    df=pd.DataFrame(result)
    
    return df,counter

## In order to convert number to datetime

In [42]:
def get_time(data):
    result={}
    start_date = "2016/03" 
    stop_date="2019/03"
    start=datetime.strptime(start_date,"%Y/%m")
    stop=datetime.strptime(stop_date,"%Y/%m")
    dates=[]
    arr_smonth=[]
    arr_emonth=[]
    for number in data:
        smonth = start +  relativedelta(months=number)
        emonth=smonth+ relativedelta(months=1)
        arr_smonth.append(smonth.strftime('%Y/%m')) # Return a string representing the date and time
        arr_emonth.append(emonth.strftime('%Y/%m')) 
    result={
        'from':arr_smonth,
        'end':arr_emonth
    }
    
    return result

## Getting the date and count of pos and neg

In [43]:

def get_count(data,path):
    data=get_time(data)
    date=list()
    pos=list()
    neg=list()
    res=dict()
   
    for i in tqdm(range(len(data['from'])), desc='[*] Counting...', dynamic_ncols=True):
        emo_df,counter=emo_classifier(date_start=data['from'][i],date_end=data['end'][i],data=path)
        date.append(data['from'][i])
        pos.append(counter[1])
        neg.append(counter[-1])
    res={
        "date":date,
        "pos":pos,
        "neg":neg
    }
    df=pd.DataFrame(res)
    return df

## China

In [44]:
china_df=get_count(data=[0, 9, 12, 17, 20, 22, 24, 29, 33],path='./data/China.csv')
china_df

[*] Counting...: 100%|██████████| 9/9 [03:00<00:00, 22.98s/it]


,date,pos,neg
0,2016/03,56,27
1,2016/12,75,57
2,2017/03,88,52
3,2017/08,106,43
4,2017/11,95,44
5,2018/01,80,22
6,2018/03,119,57
7,2018/08,106,76
8,2018/12,127,98


## Brazil

In [45]:
import warnings
warnings.filterwarnings('ignore')
brazil=[0, 5, 6, 9, 12, 16, 17, 20, 25, 27, 29, 30, 31, 35]
brazi_df=get_count(data=brazil,path='./data/Brazil.csv')
brazi_df

[*] Counting...: 100%|██████████| 14/14 [01:00<00:00,  4.22s/it]


,date,pos,neg
0,2016/03,7,1
1,2016/08,4,1
2,2016/09,5,0
3,2016/12,2,1
4,2017/03,7,4
5,2017/07,4,3
6,2017/08,8,1
7,2017/11,1,4
8,2018/04,4,0
9,2018/06,2,3


## India

In [46]:
india=[2, 3, 4, 6, 9, 12, 20, 24, 25, 29, 30, 31, 33]
india_df=get_count(data=india,path='./data/India.csv')
india_df

[*] Counting...: 100%|██████████| 13/13 [01:06<00:00,  4.73s/it]


,date,pos,neg
0,2016/05,12,3
1,2016/06,10,0
2,2016/07,9,4
3,2016/09,8,2
4,2016/12,16,14
5,2017/03,17,5
6,2017/11,7,4
7,2018/03,6,4
8,2018/04,6,3
9,2018/08,6,4


## Russia

In [47]:
russia=[0, 5, 6, 7, 9, 12, 19, 20, 24, 32, 35]
russia_df=get_count(data=russia,path='./data/Russia.csv')
russia_df

[*] Counting...: 100%|██████████| 11/11 [00:45<00:00,  4.03s/it]


,date,pos,neg
0,2016/03,2,0
1,2016/08,4,0
2,2016/09,5,1
3,2016/10,1,2
4,2016/12,12,2
5,2017/03,5,0
6,2017/10,2,2
7,2017/11,2,2
8,2018/03,4,1
9,2018/11,2,3
